<a href="https://colab.research.google.com/github/Vakhranev/Pushkina/blob/main/%D0%9C%D1%91%D1%80%D0%B4%D0%B6%20%D1%81%20%D0%B3%D0%BB%D0%BE%D1%81%D1%81%D0%B0%D1%80%D0%B8%D1%8F%D0%BC%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import zipfile
import os
import pandas as pd
import re

# Распаковываем архив
with zipfile.ZipFile("учебники.zip", "r") as zip_ref:
    zip_ref.extractall("учебники_extracted")

# Папки с учебниками
folders = ["учебники_extracted/books/world", "учебники_extracted/books/rf"]

data = []

# Обрабатываем файлы
for folder in folders:
    for filename in os.listdir(folder):
        match = re.match(r"(\d+)", filename)
        if match:
            grade = match.group(1)  # Извлекаем номер класса
            filepath = os.path.join(folder, filename)

            with open(filepath, "r", encoding="utf-8") as file:
                for line in file:
                    term = line.strip().lower()  # Приводим к нижнему регистру
                    term = re.sub(r"[\(\)\[\],]", "", term)  # Удаляем запятые и скобки
                    term = term.replace(" ", "_")  # Заменяем пробелы на "_"

                    if term:
                        data.append([term, grade])

# Создаём DataFrame
df = pd.DataFrame(data, columns=["Термины глоссариев", "Классы глоссариев"])

# Сохраняем в Excel
df.to_excel("glossary_terms.xlsx", index=False)

print("Файл glossary_terms.xlsx сохранён.")

Файл glossary_terms.xlsx сохранён.


In [8]:
import pandas as pd

# Загружаем файл
glossary_terms = pd.read_excel("glossary_terms.xlsx")

# Приводим столбец "Классы глоссариев" к числовому типу
glossary_terms["Классы глоссариев"] = pd.to_numeric(glossary_terms["Классы глоссариев"], errors="coerce")

# Удаляем дубликаты, оставляя запись с минимальным классом
glossary_terms = glossary_terms.sort_values("Классы глоссариев").drop_duplicates(subset=["Термины глоссариев"], keep="first")

# Сохраняем очищенные данные
glossary_terms.to_excel("glossary_terms_cleaned.xlsx", index=False)

print("Файл glossary_terms_cleaned.xlsx сохранён.")

Файл glossary_terms_cleaned.xlsx сохранён.


In [9]:
import pandas as pd

# Загружаем файлы
merged_terms = pd.read_excel("merged_terms.xlsx")
glossary_terms = pd.read_excel("glossary_terms_cleaned.xlsx")

# Пробуем объединить по "Начальные формы" и "Термины глоссариев"
merged_df = merged_terms.merge(glossary_terms, left_on="Начальные формы", right_on="Термины глоссариев", how="outer")

# Если не получилось, пробуем по "termin"
if merged_df.empty:
    merged_df = merged_terms.merge(glossary_terms, left_on="termin", right_on="Термины глоссариев", how="outer")

# Сохраняем результат
merged_df.to_excel("final_merged.xlsx", index=False)

print("Файл final_merged.xlsx сохранён.")

Файл final_merged.xlsx сохранён.
